In [5]:
! nvidia-smi


Thu Dec 14 04:38:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# importing hugging face all requirements
! pip install datasets transformers[sentencepiece] sacrebleu -q

In [7]:
! pip install --upgrade accelerate
! pip uninstall -y transformers accelerate
! pip install transformers accelerate

Found existing installation: transformers 4.36.0
Uninstalling transformers-4.36.0:
  Successfully uninstalled transformers-4.36.0
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
  Using cached transformers-4.36.0-py3-none-any.whl (8.2 MB)
  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)


In [9]:
# import all important libararies
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer


In [10]:
# import the model checkpoint
model_checkpoint="Helsinki-NLP/opus-mt-en-hi"

In [11]:
# load the datasets
raw_datasets=load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [12]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [13]:
raw_datasets["train"][1]

{'translation': {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}}

In [14]:
raw_datasets["train"][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [16]:
# Download the tokenizer from huggingface
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [17]:
# tested some sentences using tokenizer
tokenizer("Hello,this is the sentence!")

{'input_ids': [12110, 2, 4470, 23, 4, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

this is the embedding and attention mask,and attentiton mask all show one because give attention all word of importance


In [18]:
tokenizer(["Hello,this is a sentence!","This is a another sentence."])

{'input_ids': [[12110, 2, 4470, 23, 19, 8800, 61, 0], [239, 23, 19, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

In [19]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(["एक्सेर्साइसर पहुंचनीयता अन्वेषक"]))

{'input_ids': [[26618, 16155, 346, 33383, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


max ids show  sub-level tokenizer

In [20]:
max_input_length=128
max_target_length=128
source_lang="en"
target_lang="hi"
def preprocess_function(examples):
  inputs=[ex[source_lang] for ex in examples["translation"]]
  targets=[ex[target_lang] for ex in examples["translation"]]
  model_inputs=tokenizer(inputs,max_length=max_input_length, truncation=True)

  # setup the tokenizer for the target
  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs



In [22]:
# apply inside my every datasets
tokenized_datasets=raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [27]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2507
    })
})

In [23]:
# Lets load the model
model= TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [24]:
batch_size=16
learning_rate=2e-5
weight_decay=0.01
num_train_epochs=1

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [26]:
generation_data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128 )

In [28]:
# load my training datasets
train_dataset=model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [30]:
# also load my validation datasets
validation_dataset=model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,

)

In [32]:
# also load my generation datasets
generation_dataset=model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator,

)

generation datasets like it will helps you give you some matrix


In [33]:
optimizer=AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [34]:
model.fit(train_dataset, validation_data = validation_dataset, epochs=1)

156/156 [==============================] - 93s 369ms/step - loss: 3.7653 - val_loss: 3.9252


# Inferencing

In [35]:
# save the model
model.save_pretrained("tf_model/")

In [36]:
# loading my tokenizer and loading my model
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)
model=TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [41]:
# translation
input_text="My name is Sultan.I love learn new technology"
tokenized=tokenizer([input_text], return_tensors="np")
out=model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   500   179     5  6417  7423   314    40   208  1438 12549  6737
   1346     5    40     0]], shape=(1, 16), dtype=int32)


In [42]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))

मेरा नाम है सुजान। मुझे नई तकनीक सीखना पसंद है।


In [43]:
# translation
input_text1="Ronaldo is the best player in the world."
tokenized=tokenizer([input_text1], return_tensors="np")
out=model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[61949  3353 44434   917   463    11   505   470  4805     5    40     0]], shape=(1, 12), dtype=int32)


In [44]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))

रॉनल्डो दुनिया में सबसे अच्छा खिलाड़ी है।


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


train multiple epochs then model better performance